In [6]:
import pandas as pd
from influxdb_client import InfluxDBClient
from pulsar_data_collection.database_connectors.influxdb import Influxdb
import os

# Set up InfluxDB connection
token = os.getenv("DOCKER_INFLUXDB_INIT_ADMIN_TOKEN")
org = os.getenv("DOCKER_INFLUXDB_INIT_ORG")
bucket = os.getenv("DOCKER_INFLUXDB_INIT_BUCKET")
url = 'http://influxdb:8086'

# Create an instance of Influxdb from your package
influxdb = Influxdb().get_database_actions()

db_login = {
    "url": url,
    "token": token,
    "org": org,
    "bucket_name": bucket
}

# Test InfluxDB connection
db_connection = influxdb.make_connection(**db_login)
if db_connection:
    print("Connection successful")
else:
    print("Connection failed")

Connection successful


In [7]:
# Print the existing bucket names
client = InfluxDBClient(url=url, token=token, org=org)

buckets_api = client.buckets_api()
buckets = buckets_api.find_buckets()

for bucket in buckets.buckets:
    print(bucket.name)

_tasks
_monitoring
demo


In [2]:
from gen_data_and_simulate_drift import DriftIntensity, DriftSimulator, GenerateFakeData
from training_script import Classifier

ModuleNotFoundError: No module named 'gen_data_and_simulate_drift'

In [ ]:
pokemon_test_data='pokemon.csv'
SAMPLE_SIZE=1000
target = 'Legendary'
genertor_fake_data = GenerateFakeData(path_ref_data=pokemon_test_data, sample_size=SAMPLE_SIZE, target=target)
sampled_data = genertor_fake_data.get_dataclass_sampling()

In [ ]:
# if the task is classification
pok_classifier = Classifier(df_train=sampled_data.train_data,
            num_features=sampled_data.list_num_col,
            cat_features=None,
            target=target,
            pkl_file_path=f'class_{target}_model.pkl')
pok_classifier.train()
pok_classifier.serialize()